In [93]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn import metrics 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP assignment 2/TRAIN

/content/drive/MyDrive/Colab Notebooks/NLP assignment 2/TRAIN


In [5]:
df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP assignment 2/TRAIN/labeled_10k.csv")

In [6]:
df.head()

,hmid,moment,concepts,agency,social,age,country,gender,married,parenthood,reflection,duration
0,27674,I was happy when my son got 90% marks in his e...,education|family,no,yes,29.0,IND,m,married,y,24h,half_a_day
1,27685,went to movies with my friends it was fun,entertainment,yes,yes,29.0,IND,m,single,y,24h,half_a_day
2,27691,A hot kiss with my girl friend last night made...,romance,yes,yes,25.0,IND,m,married,y,24h,at_least_one_hour
3,27701,My son woke me up to a fantastic breakfast of ...,family|food,no,yes,79,USA,f,widowed,y,24h,all_day_im_still_feeling_it
4,27712,My older daughter keeps patting my younger dau...,family,no,yes,30,USA,f,married,y,24h,a_few_moment


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10560 entries, 0 to 10559
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hmid        10560 non-null  int64 
 1   moment      10560 non-null  object
 2   concepts    10560 non-null  object
 3   agency      10560 non-null  object
 4   social      10560 non-null  object
 5   age         10551 non-null  object
 6   country     10538 non-null  object
 7   gender      10554 non-null  object
 8   married     10545 non-null  object
 9   parenthood  10551 non-null  object
 10  reflection  10560 non-null  object
 11  duration    10529 non-null  object
dtypes: int64(1), object(11)
memory usage: 990.1+ KB


In [8]:
df.isnull().sum()

hmid           0
moment         0
concepts       0
agency         0
social         0
age            9
country       22
gender         6
married       15
parenthood     9
reflection     0
duration      31
dtype: int64

In [9]:
df.dropna(subset= ['age','country','gender','married','parenthood','duration'], inplace= True)

In [10]:
a= df['age'].values[df['age'].apply(lambda x: isinstance(x, str))].tolist()
regex= r'[A-Za-z]+'
regex1= r'\s+'
templist= []
for word in a:
    temp= re.sub(regex1,'a', word)
    temp1= re.sub(regex,"0", temp)
    templist.append(temp1)
tempp= []
for i in templist:
    tempp.append(int(float(i)))
df['age']= tempp

In [11]:
df.isna().sum()

hmid          0
moment        0
concepts      0
agency        0
social        0
age           0
country       0
gender        0
married       0
parenthood    0
reflection    0
duration      0
dtype: int64

In [12]:
def clean_df(corpus):
    clean_text=[]
    for line in df['moment'].values:
        regex1= r'[^A-Za-z]|[0-9]'
        regex2= r'[^A-Za-z]'
        temp1= re.sub(regex1,' ',line)
        corpus2=re.sub("\s+",' ',temp1)
        #corpus2= re.split(regex2, temp2)
        clean_text.append(corpus2.strip()) 
    return clean_text

In [13]:
clean_text =clean_df(df['moment'])
df['moment']= clean_text 

In [14]:
df['moment']

0        I was happy when my son got marks in his exami...
1                went to movies with my friends it was fun
2        A hot kiss with my girl friend last night made...
3        My son woke me up to a fantastic breakfast of ...
4        My older daughter keeps patting my younger dau...
                               ...                        
10555     My husband called me just to tell me he loved me
10556         I worked out which always makes me feel good
10557     Finally got to watch the new Resident Evil movie
10558    I got to talk to an old friend and reminisce o...
10559    I had a great meeting yesterday at work with m...
Name: moment, Length: 10485, dtype: object

In [15]:
le_df = df.drop(['moment','hmid'], axis=1)
le_df.reset_index(drop= True, inplace= True)
var_mod = le_df.columns
Label_encoder = LabelEncoder()
for i in var_mod:
    if i!= 'age':
        le_df[i] = Label_encoder.fit_transform(le_df[i])
le_df.reset_index(drop=True, inplace= True)

In [16]:
le_df_1= le_df.drop(['agency','social'],axis= 1)

In [17]:
def feature_eng(method_name,data):
    if method_name== 'BOW':
        BOW_1 = CountVectorizer()  
        vectorized= BOW_1.fit_transform(data)
    elif method_name== 'BOW_with_N_gram':
        BOW_2 = CountVectorizer(ngram_range= (1,2))
        vectorized= BOW_2.fit_transform(data)
    elif method_name== 'TFIDF':
        TFIDF_1 = TfidfVectorizer()
        vectorized= TFIDF_1.fit_transform(data)
    elif method_name== 'TFIDF_with_N_gram':
        TFIDF_2 = TfidfVectorizer(ngram_range= (1,2))
        vectorized= TFIDF_2.fit_transform(data)
    return vectorized

In [18]:
temp_list= ['BOW', 'BOW_with_N_gram', 'TFIDF','TFIDF_with_N_gram']
data= df['moment'].values
for method in temp_list:
      locals()["vectorized_"+str(method)]= feature_eng(method,data)
      locals()["TSNE_"+str(method)]= TSNE(n_components=2, learning_rate='auto',init='random').fit_transform(locals()["vectorized_"+str(method)])

In [97]:
def training_model(model_name,X, y, print_info):
    if model_name == 'knn':
        model = KNeighborsClassifier(n_neighbors=7,n_jobs=-1,weights='distance',metric='euclidean')
    elif model_name == "Bagging_Classifier":
        model = BaggingClassifier(DecisionTreeClassifier(random_state= 20), 
                                               n_estimators= 500, max_samples= 100,
                                               bootstrap=True, n_jobs=-1)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y.values,train_size = 0.7)

    my_model= model.fit(X_train, y_train)
    predictions= model.predict(X_test)
    print("The accuracy for {} model on {} is: ".format(model_name, print_info),accuracy_score(y_test,predictions))
    print(metrics.classification_report(y_test, predictions))
    print("The confusion_matrix for {} model on {} is: ".format(model_name, print_info),"\n", confusion_matrix(y_test, predictions))
    return predictions

In [32]:
def plot_pred(method_name, model_name, y_df, print_info):
    temp = pd.DataFrame(method_name, columns= ['X','Y'])
    final_X_df = pd.concat([temp,le_df_1], axis= 1)
    pred= training_model(model_name,final_X_df, y_df,print_info )
    return pred

### Model evaluations for 'Agency' target variable using KNN on BOW and TFIDF with and without N-grams

In [98]:
knn_BOW_pred= plot_pred(TSNE_BOW, 'knn', le_df['agency'], 'TSNE_BOW')

The accuracy for knn model on TSNE_BOW is:  0.77177368086459
              precision    recall  f1-score   support

           0       0.58      0.43      0.50       814
           1       0.82      0.89      0.85      2332

    accuracy                           0.77      3146
   macro avg       0.70      0.66      0.67      3146
weighted avg       0.76      0.77      0.76      3146

The confusion_matrix for knn model on TSNE_BOW is:  
 [[ 354  460]
 [ 258 2074]]


In [99]:
knn_BOW_ngram_pred= plot_pred(TSNE_BOW_with_N_gram, 'knn', le_df['agency'], 'TSNE_BOW_with_N_gram')

The accuracy for knn model on TSNE_BOW_with_N_gram is:  0.7536554354736172
              precision    recall  f1-score   support

           0       0.52      0.39      0.44       801
           1       0.81      0.88      0.84      2345

    accuracy                           0.75      3146
   macro avg       0.66      0.63      0.64      3146
weighted avg       0.73      0.75      0.74      3146

The confusion_matrix for knn model on TSNE_BOW_with_N_gram is:  
 [[ 310  491]
 [ 284 2061]]


In [100]:
knn_TFIDF_pred= plot_pred(TSNE_TFIDF, 'knn', le_df['agency'], 'TSNE_TFIDF')

The accuracy for knn model on TSNE_TFIDF is:  0.736808645899555
              precision    recall  f1-score   support

           0       0.49      0.35      0.41       817
           1       0.79      0.87      0.83      2329

    accuracy                           0.74      3146
   macro avg       0.64      0.61      0.62      3146
weighted avg       0.71      0.74      0.72      3146

The confusion_matrix for knn model on TSNE_TFIDF is:  
 [[ 285  532]
 [ 296 2033]]


In [101]:
knn_TFIDF_ngram_pred= plot_pred(TSNE_TFIDF_with_N_gram, 'knn', le_df['agency'], 'TSNE_TFIDF_with_N_gram')

The accuracy for knn model on TSNE_TFIDF_with_N_gram is:  0.7507946598855689
              precision    recall  f1-score   support

           0       0.53      0.40      0.45       820
           1       0.80      0.88      0.84      2326

    accuracy                           0.75      3146
   macro avg       0.67      0.64      0.65      3146
weighted avg       0.73      0.75      0.74      3146

The confusion_matrix for knn model on TSNE_TFIDF_with_N_gram is:  
 [[ 325  495]
 [ 289 2037]]


###Model evaluations for 'Agency' target variable using Bagging_Classifier on BOW and TFIDF with and without N-grams

In [102]:
Bagging_Classifier_BOW_pred= plot_pred(TSNE_BOW, 'Bagging_Classifier', le_df['agency'], 'TSNE_BOW')

The accuracy for Bagging_Classifier model on TSNE_BOW is:  0.7708200890019072
              precision    recall  f1-score   support

           0       0.65      0.22      0.33       803
           1       0.78      0.96      0.86      2343

    accuracy                           0.77      3146
   macro avg       0.72      0.59      0.60      3146
weighted avg       0.75      0.77      0.73      3146

The confusion_matrix for Bagging_Classifier model on TSNE_BOW is:  
 [[ 178  625]
 [  96 2247]]


In [103]:
Bagging_Classifier_BOW_ngram_pred= plot_pred(TSNE_BOW_with_N_gram, 'Bagging_Classifier', le_df['agency'], 'TSNE_BOW_with_N_gram')

The accuracy for Bagging_Classifier model on TSNE_BOW_with_N_gram is:  0.7523839796567069
              precision    recall  f1-score   support

           0       0.67      0.07      0.13       808
           1       0.75      0.99      0.86      2338

    accuracy                           0.75      3146
   macro avg       0.71      0.53      0.49      3146
weighted avg       0.73      0.75      0.67      3146

The confusion_matrix for Bagging_Classifier model on TSNE_BOW_with_N_gram is:  
 [[  57  751]
 [  28 2310]]


In [104]:
Bagging_Classifier_TFIDF_pred= plot_pred(TSNE_TFIDF, 'Bagging_Classifier', le_df['agency'], 'TSNE_TFIDF')

The accuracy for Bagging_Classifier model on TSNE_TFIDF is:  0.7476160203432931
              precision    recall  f1-score   support

           0       0.63      0.05      0.10       812
           1       0.75      0.99      0.85      2334

    accuracy                           0.75      3146
   macro avg       0.69      0.52      0.48      3146
weighted avg       0.72      0.75      0.66      3146

The confusion_matrix for Bagging_Classifier model on TSNE_TFIDF is:  
 [[  44  768]
 [  26 2308]]


In [105]:
Bagging_Classifier_TFIDF_ngram_pred= plot_pred(TSNE_TFIDF_with_N_gram, 'Bagging_Classifier', le_df['agency'], 'TSNE_TFIDF_with_N_gram')

The accuracy for Bagging_Classifier model on TSNE_TFIDF_with_N_gram is:  0.7469802924348379
              precision    recall  f1-score   support

           0       0.76      0.05      0.10       825
           1       0.75      0.99      0.85      2321

    accuracy                           0.75      3146
   macro avg       0.76      0.52      0.47      3146
weighted avg       0.75      0.75      0.65      3146

The confusion_matrix for Bagging_Classifier model on TSNE_TFIDF_with_N_gram is:  
 [[  42  783]
 [  13 2308]]


### Model evaluations for 'Social' target variable using KNN on BOW and TFIDF with and without N-grams

In [106]:
knn_BOW_pred_social= plot_pred(TSNE_BOW, 'knn', le_df['social'], 'TSNE_BOW')

The accuracy for knn model on TSNE_BOW is:  0.8102352193261284
              precision    recall  f1-score   support

           0       0.78      0.83      0.80      1471
           1       0.84      0.79      0.82      1675

    accuracy                           0.81      3146
   macro avg       0.81      0.81      0.81      3146
weighted avg       0.81      0.81      0.81      3146

The confusion_matrix for knn model on TSNE_BOW is:  
 [[1222  249]
 [ 348 1327]]


In [107]:
knn_BOW_ngram_pred_social= plot_pred(TSNE_BOW_with_N_gram, 'knn', le_df['social'], 'TSNE_BOW_with_N_gram')

The accuracy for knn model on TSNE_BOW_with_N_gram is:  0.7857596948506039
              precision    recall  f1-score   support

           0       0.75      0.83      0.79      1494
           1       0.83      0.75      0.79      1652

    accuracy                           0.79      3146
   macro avg       0.79      0.79      0.79      3146
weighted avg       0.79      0.79      0.79      3146

The confusion_matrix for knn model on TSNE_BOW_with_N_gram is:  
 [[1235  259]
 [ 415 1237]]


In [108]:
knn_TFIDF_pred_social= plot_pred(TSNE_TFIDF, 'knn', le_df['social'], 'TSNE_TFIDF')

The accuracy for knn model on TSNE_TFIDF is:  0.7768595041322314
              precision    recall  f1-score   support

           0       0.75      0.79      0.77      1470
           1       0.81      0.76      0.78      1676

    accuracy                           0.78      3146
   macro avg       0.78      0.78      0.78      3146
weighted avg       0.78      0.78      0.78      3146

The confusion_matrix for knn model on TSNE_TFIDF is:  
 [[1167  303]
 [ 399 1277]]


In [109]:
knn_TFIDF_ngram_pred_social= plot_pred(TSNE_TFIDF_with_N_gram, 'knn', le_df['social'], 'TSNE_TFIDF_with_N_gram')

The accuracy for knn model on TSNE_TFIDF_with_N_gram is:  0.7838525111252383
              precision    recall  f1-score   support

           0       0.75      0.82      0.78      1492
           1       0.82      0.75      0.79      1654

    accuracy                           0.78      3146
   macro avg       0.79      0.79      0.78      3146
weighted avg       0.79      0.78      0.78      3146

The confusion_matrix for knn model on TSNE_TFIDF_with_N_gram is:  
 [[1224  268]
 [ 412 1242]]


### Model evaluations for 'Social' target variable using Bagging_Classifier on BOW and TFIDF with and without N-grams

In [110]:
Bagging_Classifier_BOW_pred_social = plot_pred(TSNE_BOW, 'Bagging_Classifier', le_df['social'], 'TSNE_BOW')

The accuracy for Bagging_Classifier model on TSNE_BOW is:  0.7676414494596313
              precision    recall  f1-score   support

           0       0.74      0.79      0.76      1476
           1       0.80      0.75      0.77      1670

    accuracy                           0.77      3146
   macro avg       0.77      0.77      0.77      3146
weighted avg       0.77      0.77      0.77      3146

The confusion_matrix for Bagging_Classifier model on TSNE_BOW is:  
 [[1164  312]
 [ 419 1251]]


In [111]:
Bagging_Classifier_BOW_ngram_pred_social= plot_pred(TSNE_BOW_with_N_gram, 'Bagging_Classifier', le_df['social'], 'TSNE_BOW_with_N_gram')

The accuracy for Bagging_Classifier model on TSNE_BOW_with_N_gram is:  0.7393515575333757
              precision    recall  f1-score   support

           0       0.70      0.77      0.74      1475
           1       0.78      0.71      0.74      1671

    accuracy                           0.74      3146
   macro avg       0.74      0.74      0.74      3146
weighted avg       0.74      0.74      0.74      3146

The confusion_matrix for Bagging_Classifier model on TSNE_BOW_with_N_gram is:  
 [[1142  333]
 [ 487 1184]]


In [112]:
Bagging_Classifier_TFIDF_pred_social= plot_pred(TSNE_TFIDF, 'Bagging_Classifier', le_df['social'], 'TSNE_TFIDF')

The accuracy for Bagging_Classifier model on TSNE_TFIDF is:  0.7399872854418309
              precision    recall  f1-score   support

           0       0.71      0.77      0.74      1489
           1       0.77      0.72      0.74      1657

    accuracy                           0.74      3146
   macro avg       0.74      0.74      0.74      3146
weighted avg       0.74      0.74      0.74      3146

The confusion_matrix for Bagging_Classifier model on TSNE_TFIDF is:  
 [[1143  346]
 [ 472 1185]]


In [113]:
Bagging_Classifier_TFIDF_ngram_pred_social= plot_pred(TSNE_TFIDF_with_N_gram, 'Bagging_Classifier', le_df['social'], 'TSNE_TFIDF_with_N_gram')

The accuracy for Bagging_Classifier model on TSNE_TFIDF_with_N_gram is:  0.7546090273363001
              precision    recall  f1-score   support

           0       0.70      0.81      0.75      1453
           1       0.81      0.71      0.76      1693

    accuracy                           0.75      3146
   macro avg       0.76      0.76      0.75      3146
weighted avg       0.76      0.75      0.75      3146

The confusion_matrix for Bagging_Classifier model on TSNE_TFIDF_with_N_gram is:  
 [[1180  273]
 [ 499 1194]]
